# Introduction to Remote Sensing with Python

>"...the time is now right and urgent to apply space technology towards the solution of many pressing natural resources problems being compounded by population and industrial growth."

- [Stewart Udall, Secretary of the Interior, September 21, 1966](https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/1966.09.21-DOI-Udall-Earth%20Resources%20Studied%20From%20Space.pdf)

## A remote sensing crash course

<img src="images/remote.png" width=600>

https://youtu.be/3iaFzafWJQE


## Landsat

Landsat is the first "civilian Earth observation satellite" launched in 1972, a collaboration between the Department of the Interior, NASA, and the Department of Agriculture.

At over 40 years, the Landsat series of satellites provides the longest temporal record of moderate resolution multispectral data of the Earth’s surface on a global basis. The Landsat record has remained remarkably unbroken, proving a unique resource to assist a broad range of specialists in managing the world’s food, water, forests, and other natural resources for a growing world population. It is a record unmatched in quality, detail, coverage, and value. Source: USGS

- [NASA's Landsat page](https://www.nasa.gov/mission_pages/landsat/overview/index.html)
- [USGS's Landsat page](https://www.usgs.gov/core-science-systems/nli/landsat)

<img src="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/full_width/public/thumbnails/image/5262-Landsat-timeline-FNL-2.jpg">

<a href="https://youtu.be/bmoYMzGBN5s"><img src="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/atom_page_medium/public/thumbnails/image/Landsat_8_%28LDCM%29_Satellite_over_Earth%2C%20Wiki%20Commons.jpg">
</a>


## How to access landsat data

- [USGS Earth Explorer](https://earthexplorer.usgs.gov/)
- [Amazon AWS Open Data](https://registry.opendata.aws/landsat-8/)
- [Google Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/landsat-8)

## Finding satellite image of the Bobcat Fire

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Bobcat_Fire%2C_Los_Angeles%2C_San_Gabriel_Mountains.jpg/800px-Bobcat_Fire%2C_Los_Angeles%2C_San_Gabriel_Mountains.jpg">

The Bobcat Fire was a fire that started on September 6, 2020 as part of the 2020 California wildfire season. By December 18, it was fully contained and had burned 115,796 acres (46,861 ha) in the central San Gabriel Mountains, in and around the Angeles National Forest. It was one of the largest fires on record in Los Angeles County to date.

- https://en.wikipedia.org/wiki/Bobcat_Fire

In this lab, we will do the following:

- Use Google Earth Engine's Python API
- define an AOI (area of interest) in the Central San Gabriel Mountains
- import multiple Landsat images from the dates before and after the Bobcat fire
- determine which images are best for analysis
- create various NDVI map outputs to assess the amount of fire damage

## Import libraries

In [1]:
# the regulars
import geopandas as gpd
import matplotlib.pyplot as plt

# work with dates
from datetime import datetime as dt

# allow images to display
from IPython.display import Image

# earth engine
import ee

/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.1-CAPI-1.13.3) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


## Authenticate Earth Engine

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

KeyboardInterrupt: Interrupted by user

For this lab, we will use Google Earth Engine's "USGS Landsat 8 Surface Reflectance Tier 1"

- [EE Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR)

You can also:

- `.filterBounds()` method allows you to filter by location
- `.filterDate()` allows you to filter by date

## Define filters

In [4]:
# coordinates of the bobcat fire
lat = 34.289
lon = -117.977

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2020-08-01'

# end date
end_date = '2020-12-31'

## Get Landsat 8 data

In [5]:
# get the satellite data
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [6]:
# how many images did we get?
print('Total number:', landsat.size().getInfo())

Total number: 8


In [7]:
# information about the first image in our collection
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7751, 7891],
   'crs': 'EPSG:32611',
   'crs_transform': [30, 0, 265785, 0, -30, 3948615]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7751, 7891],
   'crs': 'EPSG:32611',
   'crs_transform': [30, 0, 265785, 0, -30, 3948615]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7751, 7891],
   'crs': 'EPSG:32611',
   'crs_transform': [30, 0, 265785, 0, -30, 3948615]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [7751, 7891],
   'crs': 'EPSG:32611',
   'crs_transform': [30, 0, 265785, 0, -30, 3948615]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'in

In [8]:
# what about cloud cover of our first image?
landsat.first().get('CLOUD_COVER').getInfo()

15.31

In [9]:
# when was this image taken?
date = ee.Date(landsat.first().get('system:time_start')).getInfo()['value']
time = date/1000
date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
date

'2020-08-04 18:28:17'

## Bands

<img src="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/thumbnails/image/Landsat%208%20band%20designations.jpg" width=800>

In [10]:
# what bands did we get?
landsat.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [11]:
# put the images in a list
landsat_list = landsat.toList(landsat.size());

## Display satellite image

In [12]:
# set some parameters for image
parameters = {
                'min': 0,
                'max': 2500,
                'dimensions': 400,
                'bands': ['B4', 'B3', 'B2']
             }

In [13]:
# loop through each image and display it
for i in range(landsat.size().getInfo()):

    # when was this image taken?
    date = ee.Date(ee.Image(landsat_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
    
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))
    

Image # 0 2020-08-04 18:28:17 Cloud cover: 15.31


Image # 1 2020-08-20 18:28:23 Cloud cover: 0.17


Image # 2 2020-09-05 18:28:31 Cloud cover: 1.06


Image # 3 2020-09-21 18:28:36 Cloud cover: 7.32


Image # 4 2020-10-07 18:28:39 Cloud cover: 1.63


Image # 5 2020-10-23 18:28:40 Cloud cover: 39.65


Image # 6 2020-11-24 18:28:39 Cloud cover: 0.06


Image # 7 2020-12-10 18:28:41 Cloud cover: 39.43


## Selecting images, zooming in
Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study. Ideally, we want to have images for before and after the fire to be able to assess the level of damage.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by appying a 20km buffer around our POI.

In [14]:
# create a list of images we want (before, during, after)
landsat_sequence = [1,3,4]

In [15]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(20000) # meters

In [16]:
parameters = {'min': 0,
              'max': 2500,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region':roi
              }

In [17]:
for i in landsat_sequence:
    # when was this image taken?
    date = ee.Date(ee.Image(landsat_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 1 2020-08-20 18:28:23 Cloud cover: 0.17


Image # 3 2020-09-21 18:28:36 Cloud cover: 7.32


Image # 4 2020-10-07 18:28:39 Cloud cover: 1.63


<div class="alert alert-info">
Take a moment to inspect the three images above. What do they tell you? What do they NOT tell you?
</div>

## Normalized Difference Vegetation Index (NDVI)

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

<img src="https://eos.com/wp-content/uploads/2020/11/how-NDVI-works-EN.jpg" width=800>

- [EOS](https://eos.com/ndvi/)

In [18]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [19]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Date(ee.Image(landsat_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
    
    # print some information
    print('Image #',i,date)
    
    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['B5', 'B4']).getThumbUrl(ndvi_parameters)))

Image # 1 2020-08-20 18:28:23


Image # 3 2020-09-21 18:28:36


Image # 4 2020-10-07 18:28:39


## Folium

Of course, we can't end the lab without an interactive map.

In [20]:
import folium

In [21]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [22]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Date(ee.Image(landsat_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['B5', 'B4']), 
                        ndvi_parameters, 
                        name=date)

# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

## Save the folium map as an html file

In [23]:
my_map.save('bobcat.html')

# Resources

- [Google's Python Tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)
- [Earth Lab](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/)
    